# 起票されたケースの解決策を示すAgentを作成する(標準時間:60分)

## 本ノートブックの目的：Databricks における AI Agent 
Q1. RAG Agent の構築<br>
Q2. Driver Notebookの実行とReview Apps による評価<br>
Q3. Databricks Apps との連携

## Q1. RAG Agent の構築(標準時間：10分)

Databricks の AI Playground を使用して、`openhack-gpt-4o` エンドポイントと `manual_retriever` ツールを登録し、ringo computer のマニュアルに関する質問に答えるエージェントを作成する手順は以下の通りです:

1. Databricks ワークスペースにログインし、左側のナビゲーションから [Machine Learning] > [Playground] を選択します。

2. AI Playground の画面で、左上のドロップダウンメニューから `openhack-gpt-4o` エンドポイントを選択します。
![](/Workspace/Users/shotaro.kotani@nssoldlp.onmicrosoft.com/openhack2025/contents_with_answer/dev_shotkotani/data/img/endpoint_selection.png)
3. [ツール] セクションを探し、`manual_retriever` ツールを選択します。このツールが表示されない場合は、カスタムツールとして追加する必要があります。
![](/Workspace/Users/shotaro.kotani@nssoldlp.onmicrosoft.com/openhack2025/contents_with_answer/dev_shotkotani/data/img/tool_selection.png)

4. システムプロンプトを設定します。例えば:
   "あなたは ringo computer のマニュアルに関する質問に答えるエキスパートです。manual_retriever ツールを使用して、適切な情報を取得し、ユーザーの質問に答えてください。"
   ![](/Workspace/Users/shotaro.kotani@nssoldlp.onmicrosoft.com/openhack2025/contents_with_answer/dev_shotkotani/data/img/export_notebook.png)

5. チャットインターフェースを使用して、エージェントとの対話をテストします。以下のようなサンプル質問を試してみましょう:
   - "ringo computer の電源の入れ方を教えてください。"
   - "ringo computer のメモリを増設する方法は？"
   - "ringo computer のOSをアップデートする手順を説明してください。"

6. エージェントの応答を確認し、必要に応じてシステムプロンプトや設定を調整します。

7. エージェントの動作に満足したら、画面右上の [エクスポート] ボタンをクリックします。

8. エクスポートされたノートブックには以下のファイルが含まれます:
   - `agent` ノートブック: LangChain を使用してエージェントを定義する Python コード
   - `driver` ノートブック: エージェントのログ記録、トレース、登録、デプロイ用のコード
   - `config.yml`: エージェントの設定情報

9. エクスポートされたノートブックを開き、コードを確認します。必要に応じて調整を行います。

10. `driver` ノートブックを実行して、エージェントをモデルサービングエンドポイントにデプロイします。



## Q2. Driver Notebookの実行とReview Apps による評価(標準時間：10分)
AI Playgroundで作成されたdriver notebookは、エージェントのログ記録、評価、登録、およびデプロイを行うための重要なコンポーネントです。

### `driver` ノートブックの流れ
- Mosaic AIエージェントフレームワークを使用してPythonエージェントの処理を行います
- エージェントをMLflowにログ記録します
- エージェントをUnity Catalogに登録します
- エージェントをモデルサービングエンドポイントにデプロイします
- エージェントを評価するReview Appをデプロイして、ステークホルダーに配布できます

### メリット
- 自動生成されるため、開発時間を短縮できます
- エージェントのライフサイクル管理を一元化し、効率的に行えます
- MLflowとの統合により、実験管理や再現性の確保が容易になります
- Unity Catalogへの登録により、エージェントの共有と再利用が促進されます
- モデルサービングエンドポイントへのデプロイにより、本番環境での利用が簡単になります

### 注意
エクスポートされたコードは AI Playground セッションとは異なる動作をする可能性があります。<br>
1. `driver` ノートブックのセル3でインストールするlanggraphのバージョンを以下のように固定してください
```langgraph==0.2.74```

2. セル11においてlanggraphバージョンを固定してください。
```
pip_requirements=[
            "mlflow",
            "langchain",
            "langgraph==0.2.74",
            "databricks-langchain",
            "unitycatalog-langchain[databricks]",
            "pydantic",
        ]
```

3. セル11において、mlflowのモデルが権限を持つ必要があるリソースを指定する必要があります。`mlflow.models.resources`のモジュールからエンドポイントとベクトルサーチインデックスを指定します。
```
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint, DatabricksVectorSearchIndex
```
4. セル11において、DatabricksVectorSearchIndex のリソースを追加します。カタログとスキーマを埋めましょう。
```
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME), 
    DatabricksServingEndpoint(endpoint_name="openhack-text-embedding-ada-002"),
    DatabricksVectorSearchIndex(index_name="{catalog}.{schema}.product_documentation_vs"),
    DatabricksFunction(function_name="{catalog}.common.manual_retriever"),
    ]
```

## Q3. Databricks Apps との連携
Databricks AppsのGUIを使用してchatbot agentをデプロイする手順は以下の通りです:

1. Databricksワークスペースにログインします。

2. 左側のナビゲーションバーで「コンピュート」をクリックし、「アプリ」タブに移動します。

3. 「新規アプリの作成」ボタンをクリックします。

4. アプリの名前を入力します。例えば「chatbot-agent」などです。

5. アプリのテンプレートとして「Chatbot」または類似のオプションを選択します。

6. アプリの設定画面で、以下の情報を入力します:
   - アプリの説明
   - 
   - 環境変数（必要に応じて）

7. エージェントのエンドポイントを指定します。

8. MLflow ChatModelを使用してagentを実装していることを確認します。これにより、Databricks AI Agent機能との互換性が確保されます。

9. 設定が完了したら、「デプロイ」または「作成」ボタンをクリックします。

10. デプロイが完了すると、アプリの詳細ページが表示されます。ここでステータスや公開URLを確認できます。

11. アプリのテストを行い、必要に応じて設定やコードを調整します。

12. 公開の準備ができたら、アクセス権限を設定し、公開URLを通じてユーザーがアプリにアクセスできるようにします。